In [6]:
import pandas as pd

# Cargar datasets (ajusta los nombres si tus archivos son diferentes)
df_antiguo = pd.read_csv("valencia_clima_unificado (2).csv", delimiter=';', on_bad_lines='skip', engine='python')

df_nuevo = pd.read_csv("nuevosdatos.csv", delimiter=',', on_bad_lines='skip', engine='python')

df_antiguo["datetime"] = pd.to_datetime(df_antiguo["datetime"], errors='coerce', dayfirst=True)

df_nuevo["datetime"] = pd.to_datetime(df_nuevo["datetime"], errors='coerce', dayfirst=True)

# Asegurarse de que la columna datetime esté bien formateada
df_antiguo["datetime"] = pd.to_datetime(df_antiguo["datetime"], errors='coerce')
df_nuevo["datetime"] = pd.to_datetime(df_nuevo["datetime"], errors='coerce')

# Concatenar, eliminar duplicados por fecha y ordenar
df_total = pd.concat([df_antiguo, df_nuevo], ignore_index=True)
df_total = df_total.drop_duplicates(subset="datetime").sort_values("datetime")
df_total = df_total.dropna(subset=["datetime"]).reset_index(drop=True)


C:\Users\Elena Chirivella\AppData\Local\Temp\ipykernel_33016\1897123531.py:10: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_nuevo["datetime"] = pd.to_datetime(df_nuevo["datetime"], errors='coerce', dayfirst=True)


In [5]:
print(df_nuevo.columns[:5])  # Muestra las primeras columnas reales


Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp'], dtype='object')


In [8]:
import pandas as pd

# 1. Cargar archivos
df_antiguo = pd.read_csv("valencia_clima_unificado (2).csv", delimiter=';', on_bad_lines='skip', engine='python')
df_nuevo = pd.read_csv("nuevosdatos.csv", delimiter=',', on_bad_lines='skip', engine='python')

# 2. Convertir fechas correctamente
df_antiguo["datetime"] = pd.to_datetime(df_antiguo["datetime"], errors='coerce', dayfirst=True)
df_nuevo["datetime"] = pd.to_datetime(df_nuevo["datetime"], errors='coerce', dayfirst=True)

# 3. Unir datasets, quitar duplicados por fecha y ordenar
df_total = pd.concat([df_antiguo, df_nuevo], ignore_index=True)
df_total = df_total.drop_duplicates(subset="datetime").sort_values("datetime")
df_total = df_total.dropna(subset=["datetime"]).reset_index(drop=True)

# 4. Crear columnas de fecha
df_total["day"] = df_total["datetime"].dt.day
df_total["month"] = df_total["datetime"].dt.month
df_total["year"] = df_total["datetime"].dt.year
df_total["weekday"] = df_total["datetime"].dt.weekday
df_total["is_weekend"] = df_total["weekday"].isin([5, 6]).astype(int)

# 5. Crear lags
lags = [1, 2, 3, 7]
for l in lags:
    df_total[f"temp_lag_{l}"] = df_total["temp"].shift(l)
    df_total[f"precip_lag_{l}"] = df_total["precip"].shift(l)
    df_total[f"humidity_lag_{l}"] = df_total["humidity"].shift(l)

# 6. Solo eliminar filas con NaN en los lags (no en todo el DataFrame)
lag_cols = [f"{var}_lag_{l}" for var in ["temp", "precip", "humidity"] for l in lags]
df_model_final = df_total.dropna(subset=lag_cols).reset_index(drop=True)

# 7. Guardar CSV para usar en Streamlit
df_model_final.to_csv("df_model_final.csv", index=False)
print("✅ Archivo 'df_model_final.csv' guardado con", len(df_model_final), "filas.")



C:\Users\Elena Chirivella\AppData\Local\Temp\ipykernel_33016\595888380.py:9: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_nuevo["datetime"] = pd.to_datetime(df_nuevo["datetime"], errors='coerce', dayfirst=True)


✅ Archivo 'df_model_final.csv' guardado con 2959 filas.


In [10]:
features = [col for col in df_model_final.columns if 'lag' in col or col in ['day', 'month', 'year', 'weekday', 'is_weekend']]

# Guardar columnas para luego usarlas en la app
import joblib
joblib.dump(features, "feature_names.pkl")
print("✅ Guardado feature_names.pkl con", len(features), "columnas")


✅ Guardado feature_names.pkl con 17 columnas
